In [12]:
import pandas as pd

from pandas import DataFrame as df
import numpy as np
import csv
from sklearn.model_selection import train_test_split
import math
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn.model_selection import cross_val_score
import pickle
from sklearn import svm
from sklearn.metrics import pairwise_distances

from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import KFold
from sklearn.feature_selection import mutual_info_regression
import helper_functions as hf

criteria = "BMI"
f = open("data/AG_new/otu_train.obj", "rb")
otu_train = pickle.load(f)
f.close()

f = open("data/AG_new/otu_test.obj", "rb")
otu_test = pickle.load(f)
f.close()

f = open("data/AG_new/qual_vecs.obj", "rb")
qual_vecs = pickle.load(f)
f.close()

f = open("data/AG_new/map_train.obj", "rb")
map_train = pickle.load(f)
f.close()

f = open("data/AG_new/map_test.obj", "rb")
map_test = pickle.load(f)
f.close()

#otu_train, otu_test, qual_vecs_clean, map_train, map_test = hf.getDataAG(criteria = criteria)
#otu_train, otu_test, qual_vecs_500, map_train, map_test = getData(qual_vec_type = "500")
#otu_train can be used for all feature selection and training. Otu test is modeling a completely independent study, as would be the usecase for this framework
otu_norm = hf.normalize(otu_train)

kf = KFold(n_splits = 5)
mses = list()
features_list = list()


In [13]:
print(map_train.shape)
print(otu_train.shape)

(9036, 527)
(9036, 12018)


In [2]:
sample_sums = otu_norm.sum(axis = 1)
otu_norm = otu_norm.loc[round(sample_sums) == 1, :]
map_train = map_train.loc[round(sample_sums) == 1, :]

np.sum(round(sample_sums) == 0)

0

In [20]:
otu_norm

OTUID,1952,2222,7366,7821,10085,10180,12564,12574,13463,13485,...,4482598,4482944,4482983,4483006,4483015,4483045,4483337,4483570,4483963,4484075
000001001.1053931,0.008354,0.000000,0.000340,0.000000,0.0,0.000000,0.000000,0.000028,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000028,0.000000,0.000000,0.000028,0.000028,0.000000
000001002.1076104,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
000001004.1076113,0.017888,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000234,0.000000
000001008.1075990,0.294969,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
000001011.1210950,0.004837,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041602,0.000193
000001018.1075878,0.000000,0.000000,0.000000,0.000000,0.0,0.002372,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
000001025.1213131,0.008243,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000150,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000150,0.000000,0.000000,0.000000
000001026.1076452,0.008139,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
000001030.1257529,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
000001030.1261848,0.000059,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000118,0.000000,0.000000,...,0.000000,0.000178,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004738,0.000000


In [7]:
map_train.shape

(9036, 527)

In [14]:
target = criteria
for train_i, test_i in kf.split(otu_norm):
    print("Fold: " + str(kf))
    print(train_i)
    X_train, X_test = otu_norm.iloc[train_i,:], otu_norm.iloc[test_i, :]
    y_train, y_test = map_train.iloc[train_i, :], map_train.iloc[test_i, :]
    y_train = np.array([float(i) for i in y_train[criteria]])
    y_test = np.array([float(i) for i in y_test[criteria]])
    
    print(X_train.shape)
    print(y_train.shape)
    k_obj = SelectKBest(mutual_info_regression, k= 2).fit(X_train, y_train)
    features = otu_norm.columns.values[k_obj.get_support()]
    
    
    print("features selected")
    m = svm.SVR(gamma = "scale")
    m.fit(X_train[features], y_train)
    print("Model fit")
    
    preds = m.predict(X_test[features])
    mse = np.mean([np.square(y_test[i] - preds[i]) for i in range(len(preds))])
    mses.append(mse)
    features_list.append(features)


Fold: KFold(n_splits=5, random_state=None, shuffle=False)
[1808 1809 1810 ... 9033 9034 9035]
(7228, 12018)
(7228,)


KeyboardInterrupt: 